In [5]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from transformers import BertTokenizer, BertModel
import torch
import pickle
import os
from typing import Dict, List, Tuple, Union, Optional

class TextEmbedder:
    """
    Clase para procesar texto utilizando modelos BERT y generar embeddings.
    """
    def __init__(self, model_name='bert-base-uncased', device=None):
        """
        Inicializa el tokenizer y modelo BERT.
        
        Args:
            model_name: Nombre del modelo BERT preentrenado a utilizar
            device: Dispositivo para ejecutar el modelo (None para autodetección)
        """
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertModel.from_pretrained(model_name)
        
        # Autodetectar dispositivo si no se especifica
        if device is None:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        else:
            self.device = device
        
        self.model.to(self.device)
        self.model.eval()  # Establecer modo evaluación
    
    def get_embeddings(self, texts: List[str], max_length: int = 128) -> np.ndarray:
        """
        Convierte una lista de textos en embeddings utilizando BERT.
        
        Args:
            texts: Lista de strings a procesar
            max_length: Longitud máxima de tokens para cada texto
            
        Returns:
            Array NumPy con los embeddings de los textos
        """
        inputs = self.tokenizer(
            texts, 
            padding=True, 
            truncation=True, 
            return_tensors='pt', 
            max_length=max_length
        )
        
        # Mover inputs al dispositivo adecuado
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = self.model(**inputs)
        
        # Extraer el vector [CLS] como representación del texto
        embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        return embeddings


class ProjectViabilityModel:
    """
    Modelo para predecir la viabilidad de proyectos utilizando características 
    textuales y numéricas.
    """
    def __init__(self):
        """Inicializa el modelo y sus componentes de preprocesamiento."""
        self.text_embedder = TextEmbedder()
        self.encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
        self.scaler = StandardScaler()
        self.model = None
        
        # Definir columnas para diferentes tipos de características
        self.categorical_cols = ['category', 'stage', 'tokenized_asset_type', 
                                'legal_status', 'impact_type']
        self.text_cols = ['summary', 'problem_addressed', 
                         'solution_proposed', 'differentiation']
        self.numeric_cols = ['estimated_total_cost_eur', 'funding_requested_eur', 
                            'expected_annual_revenue_eur', 'scalability_score']
        self.binary_cols = ['asset_existing', 'current_clients_or_pilots']
        
    def preprocess_data(self, data: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
        """
        Preprocesa los datos y extrae características para el modelo.
        
        Args:
            data: DataFrame con los datos a procesar
            
        Returns:
            Tupla con (X, y) donde X son las características y y es la variable objetivo
        """
        # Verificar que todas las columnas necesarias estén presentes
        required_cols = (self.categorical_cols + self.text_cols + 
                         self.numeric_cols + self.binary_cols)
        missing_cols = [col for col in required_cols if col not in data.columns]
        if missing_cols:
            raise ValueError(f"Faltan columnas requeridas en los datos: {missing_cols}")
        
        # Procesar variables categóricas
        categorical_features = self.encoder.transform(data[self.categorical_cols])
        
        # Procesar variables textuales
        text_features = []
        for col in self.text_cols:
            col_embeddings = self.text_embedder.get_embeddings(data[col].tolist())
            text_features.append(col_embeddings)
        text_features_combined = np.hstack(text_features)
        
        # Procesar variables numéricas
        numeric_features = self.scaler.transform(data[self.numeric_cols])
        
        # Combinar todas las características
        X = np.hstack([
            categorical_features,
            text_features_combined,
            numeric_features,
            data[self.binary_cols].values
        ])
        
        # Variable objetivo (puede ser None si no está disponible)
        y = data['viability_score'] if 'viability_score' in data.columns else None
        
        return X, y
    
    def fit(self, data: pd.DataFrame, **model_params) -> Dict:
        """
        Entrena el modelo con los datos proporcionados.
        
        Args:
            data: DataFrame con los datos de entrenamiento
            **model_params: Parámetros para el RandomForestRegressor
            
        Returns:
            Diccionario con métricas de rendimiento del entrenamiento
        """
        # Valores por defecto para los parámetros del modelo
        default_params = {
            'n_estimators': 200,
            'max_depth': 15,
            'min_samples_split': 5,
            'min_samples_leaf': 2,
            'random_state': 42
        }
        
        # Actualizar con parámetros proporcionados
        for key, value in model_params.items():
            default_params[key] = value
        
        # Ajustar transformadores con todo el conjunto de datos
        self.encoder.fit(data[self.categorical_cols])
        self.scaler.fit(data[self.numeric_cols])
        
        # Preprocesar datos
        X, y = self.preprocess_data(data)
        
        # Dividir en conjuntos de entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        
        # Crear y entrenar el modelo
        self.model = RandomForestRegressor(**default_params)
        self.model.fit(X_train, y_train)
        
        # Evaluar el modelo
        y_pred = self.model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Obtener importancia de características
        feature_importances = self.model.feature_importances_
        top_indices = np.argsort(feature_importances)[-10:]
        top_importances = {f"feature_{i}": feature_importances[i] for i in reversed(top_indices)}
        
        return {
            'mse': mse,
            'r2': r2,
            'feature_importances': top_importances
        }
    
    def predict(self, data: pd.DataFrame) -> np.ndarray:
        """
        Realiza predicciones de viabilidad para nuevos proyectos.
        
        Args:
            data: DataFrame con los datos de los proyectos a evaluar
            
        Returns:
            Array con las predicciones de viabilidad
        """
        if self.model is None:
            raise ValueError("El modelo no ha sido entrenado. Llame al método fit() primero.")
        
        # Preprocesar datos (ignorando la variable y si existe)
        X, _ = self.preprocess_data(data)
        
        # Realizar predicciones
        predictions = self.model.predict(X)
        
        return predictions
    
    def validate(self, validation_data: pd.DataFrame) -> Dict:
        """
        Valida el modelo con un conjunto de datos de validación.
        
        Args:
            validation_data: DataFrame con datos de validación
            
        Returns:
            Diccionario con métricas y resultados de la validación
        """
        if self.model is None:
            raise ValueError("El modelo no ha sido entrenado. Llame al método fit() primero.")
        
        # Realizar predicciones
        predictions = self.predict(validation_data)
        
        # Añadir predicciones al DataFrame
        results_df = validation_data.copy()
        results_df['predicted_viability'] = predictions
        
        # Si existe la columna de viabilidad real, calcular métricas
        metrics = {}
        if 'viability_score' in validation_data.columns:
            mse = mean_squared_error(validation_data['viability_score'], predictions)
            r2 = r2_score(validation_data['viability_score'], predictions)
            metrics = {'mse': mse, 'r2': r2}
        
        return {
            'results': results_df,
            'metrics': metrics,
            'predictions': predictions
        }
    
    def calculate_viability_score(self, data: pd.DataFrame) -> np.ndarray:
        """
        Calcula una puntuación de viabilidad según la fórmula predefinida.
        
        Args:
            data: DataFrame con los datos de los proyectos
            
        Returns:
            Array con las puntuaciones de viabilidad calculadas
        """
        # Factor 1: La etapa más avanzada aumenta la viabilidad
        stage_score = data['stage'].map({'Idea': 10, 'Prototipo': 20, 'MVP': 30, 'Escalando': 40})
        
        # Factor 2: La existencia de clientes aumenta la viabilidad
        client_score = data['current_clients_or_pilots'] * 15
        
        # Factor 3: Mayor escalabilidad aumenta la viabilidad
        scalability_score = data['scalability_score'] * 5
        
        # Factor 4: Mejor ratio ingresos/costos aumenta la viabilidad
        revenue_ratio = np.clip(data['expected_annual_revenue_eur'] / (data['estimated_total_cost_eur'] + 1), 0, 5) * 7
        
        # Calcular puntuación final
        viability_score = stage_score + client_score + scalability_score + revenue_ratio
        
        # Limitar los valores entre 0 y 100
        viability_score = np.clip(viability_score, 0, 100)
        
        return viability_score
    
    def save_model(self, filepath: str):
        """Guarda el modelo y sus componentes para uso futuro."""
        if self.model is None:
            raise ValueError("No hay modelo entrenado para guardar.")
        
        model_data = {
            'model': self.model,
            'encoder': self.encoder,
            'scaler': self.scaler,
            'categorical_cols': self.categorical_cols,
            'text_cols': self.text_cols,
            'numeric_cols': self.numeric_cols,
            'binary_cols': self.binary_cols
        }
        
        with open(filepath, 'wb') as f:
            pickle.dump(model_data, f)
    
    @classmethod
    def load_model(cls, filepath: str):
        """
        Carga un modelo previamente guardado.
        
        Args:
            filepath: Ruta al archivo del modelo guardado
            
        Returns:
            Instancia de ProjectViabilityModel con el modelo cargado
        """
        with open(filepath, 'rb') as f:
            model_data = pickle.load(f)
        
        # Crear nueva instancia
        instance = cls()
        
        # Cargar componentes
        instance.model = model_data['model']
        instance.encoder = model_data['encoder']
        instance.scaler = model_data['scaler']
        instance.categorical_cols = model_data['categorical_cols']
        instance.text_cols = model_data['text_cols']
        instance.numeric_cols = model_data['numeric_cols']
        instance.binary_cols = model_data['binary_cols']
        
        return instance


class DataGenerator:
    """
    Clase para generar datos sintéticos de proyectos basados en ejemplos reales.
    """
    def __init__(self, project_examples=None):
        """
        Inicializa el generador de datos con ejemplos de proyectos.
        
        Args:
            project_examples: Diccionario con ejemplos de proyectos por categoría
        """
        # Usar los ejemplos proporcionados o los predeterminados
        self.project_details = project_examples or self._get_default_examples()
        
        # Definir posibles valores para cada característica
        self.categories = list(self.project_details.keys())
        self.stages = ['Idea', 'Prototipo', 'MVP', 'Escalando']
        self.asset_types = ['Equity', 'Deuda', 'Otro']
        self.legal_statuses = ['Regulado', 'No regulado']
        self.impact_types = ['Social', 'Ambiental', 'Económico']
    
    def _get_default_examples(self):
        """
        Proporciona ejemplos predeterminados de proyectos si no se especifican.
        
        Returns:
            Diccionario con ejemplos de proyectos por categoría
        """
        return {
            'Tech': [
                {
                    'summary': "Plataforma de pagos transfronterizos que facilita la transferencia de dinero de manera rápida y segura.",
                    'problem_addressed': "Los pagos internacionales son costosos y lentos, afectando a las pequeñas empresas.",
                    'solution_proposed': "Solución que ofrece tarifas de pago transfronterizas reducidas y procesamiento rápido, utilizando blockchain.",
                    'differentiation': "Utiliza una red de blockchain descentralizada para garantizar la transparencia y baja de costos en pagos."
                },
                # ... otros ejemplos
            ],
            'Energía': [
                {
                    'summary': "Desarrollo de un sistema de energía solar de bajo costo para hogares en zonas rurales.",
                    'problem_addressed': "El acceso a energía limpia en zonas rurales es limitado y costoso.",
                    'solution_proposed': "Sistema de paneles solares con batería de almacenamiento que reduce el costo de energía en áreas rurales.",
                    'differentiation': "Baterías solares desarrolladas con tecnología avanzada que optimizan la eficiencia en lugares remotos."
                },
                # ... otros ejemplos
            ]
            # ... otras categorías
        }
    
    def generate_data(self, n: int = 500) -> pd.DataFrame:
        """
        Genera un conjunto de datos sintéticos basados en ejemplos reales.
        
        Args:
            n: Número de muestras a generar
            
        Returns:
            DataFrame con datos sintéticos
        """
        # Listas para almacenar los datos generados
        category_list = []
        summary_list = []
        problem_list = []
        solution_list = []
        differentiation_list = []
        
        # Generar datos basados en ejemplos reales
        for _ in range(n):
            # Seleccionar una categoría aleatoria
            category = np.random.choice(self.categories)
            category_list.append(category)
            
            # Seleccionar un ejemplo aleatorio de esa categoría
            example = random.choice(self.project_details[category])
            
            # Extraer textos del ejemplo
            summary_list.append(example['summary'])
            problem_list.append(example['problem_addressed'])
            solution_list.append(example['solution_proposed'])
            
            # Verificar y extraer diferenciación si existe
            if 'differentiation' in example:
                differentiation_list.append(example['differentiation'])
            else:
                differentiation_list.append("Ventaja competitiva en el mercado")
        
        # Crear DataFrame con todos los datos
        data = pd.DataFrame({
            'category': category_list,
            'stage': np.random.choice(self.stages, n),
            'summary': summary_list,
            'problem_addressed': problem_list,
            'solution_proposed': solution_list,
            'differentiation': differentiation_list,
            'tokenized_asset_type': np.random.choice(self.asset_types, n),
            'asset_existing': np.random.choice([0, 1], n),
            'legal_status': np.random.choice(self.legal_statuses, n),
            'estimated_total_cost_eur': np.random.uniform(50000, 1000000, n),
            'funding_requested_eur': np.random.uniform(10000, 500000, n),
            'expected_annual_revenue_eur': np.random.uniform(20000, 2000000, n),
            'current_clients_or_pilots': np.random.choice([0, 1], n),
            'scalability_score': np.random.randint(1, 6, n),
            'impact_type': np.random.choice(self.impact_types, n),
        })
        
        # Añadir puntuación de viabilidad
        model = ProjectViabilityModel()
        data['viability_score'] = model.calculate_viability_score(data)
        
        # Añadir algo de ruido a la viabilidad
        data['viability_score'] = data['viability_score'] + np.random.normal(0, 5, n)
        data['viability_score'] = np.clip(data['viability_score'], 0, 100)
        
        return data




# Ejemplo de uso del sistema modular
def main():
    # 1. Generar datos de entrenamiento
    print("Generando datos sintéticos...")
    data_gen = DataGenerator()
    training_data = data_gen.generate_data(n=500)
    print(f"Datos generados: {training_data.shape[0]} ejemplos")
    
    # 2. Crear y entrenar modelo
    print("\nEntrenando modelo...")
    model = ProjectViabilityModel()
    training_metrics = model.fit(training_data)
    
    # 3. Mostrar métricas de entrenamiento
    print(f"MSE en entrenamiento: {training_metrics['mse']:.2f}")
    print(f"R² en entrenamiento: {training_metrics['r2']:.2f}")
    
    print("\nCaracterísticas más importantes:")
    for feature, importance in training_metrics['feature_importances'].items():
        print(f"{feature}: {importance:.4f}")
    
    # 4. Generar datos para validación
    print("\nGenerando datos de validación...")
    validation_data = data_gen.generate_data(n=100)  # Menos ejemplos para validación
    
    # 5. Validar modelo con nuevos datos
    print("Validando modelo...")
    validation_results = model.validate(validation_data)
    
    # 6. Mostrar métricas de validación
    print(f"MSE en validación: {validation_results['metrics']['mse']:.2f}")
    print(f"R² en validación: {validation_results['metrics']['r2']:.2f}")
    
    # 7. Guardar modelo para uso futuro
    model_path = "project_viability_model.pkl"
    model.save_model(model_path)
    print(f"\nModelo guardado en: {model_path}")
    
    # 8. Ejemplo de carga y uso del modelo guardado
    print("\nCargando modelo guardado...")
    loaded_model = ProjectViabilityModel.load_model(model_path)
    
    # 9. Crear un ejemplo específico para probar
    example_project = {
        'category': ['Tech'],
        'stage': ['MVP'],
        'summary': ["Nueva plataforma de inteligencia artificial para optimización de procesos industriales"],
        'problem_addressed': ["Los procesos industriales actuales son ineficientes y generan altos costos operativos"],
        'solution_proposed': ["Software de IA que analiza procesos en tiempo real y recomienda optimizaciones"],
        'differentiation': ["Algoritmos propietarios que reducen costos operativos en un 30% más que competidores"],
        'tokenized_asset_type': ['Equity'],
        'asset_existing': [1],
        'legal_status': ['Regulado'],
        'estimated_total_cost_eur': [350000],
        'funding_requested_eur': [200000],
        'expected_annual_revenue_eur': [1200000],
        'current_clients_or_pilots': [1],
        'scalability_score': [5],
        'impact_type': ['Económico']
    }
    example_df = pd.DataFrame(example_project)
    
    # 10. Predecir viabilidad del ejemplo
    prediction = loaded_model.predict(example_df)[0]
    print(f"Predicción de viabilidad para el proyecto ejemplo: {prediction:.2f}/100")

if __name__ == "__main__":
    main()

Generando datos sintéticos...
Datos generados: 500 ejemplos

Entrenando modelo...
MSE en entrenamiento: 47.43
R² en entrenamiento: 0.85

Características más importantes:
feature_3088: 0.2102
feature_3086: 0.1911
feature_3: 0.1351
feature_3089: 0.1225
feature_3091: 0.1197
feature_2: 0.1047
feature_5: 0.0359
feature_4: 0.0262
feature_3087: 0.0231
feature_7: 0.0042

Generando datos de validación...
Validando modelo...
MSE en validación: 66.58
R² en validación: 0.79

Modelo guardado en: project_viability_model.pkl

Cargando modelo guardado...
Predicción de viabilidad para el proyecto ejemplo: 83.95/100
